##### Copyright 2021 The TensorFlow Federated Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Ajuste de agregaciones recomendadas para el aprendizaje

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/federated/tutorials/tuning_recommended_aggregators"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/federated/tutorials/tuning_recommended_aggregators.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/federated/tutorials/tuning_recommended_aggregators.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/federated/tutorials/tuning_recommended_aggregators.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table>

El módulo `tff.learning` contiene varias formas de agregar actualizaciones de modelos con la configuración predeterminada recomendada:

- `tff.learning.robust_aggregator`
- `tff.learning.dp_aggregator`
- `tff.learning.compression_aggregator`
- `tff.learning.secure_aggregator`

En este tutorial, explicamos la motivación subyacente, cómo se implementan y le sugerimos cómo personalizar su configuración. 

---

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated

In [1]:
import math
import tensorflow_federated as tff
tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

Los métodos de agregación están representados por objetos que se pueden pasar a `tff.learning.algorithms.build_weighted_fed_avg` (y a `build_unweighted_fed_avg`) como argumento de la palabra clave `model_aggregator`. De este modo, los agregadores que analizamos aquí se pueden usar directamente para modificar un [tutorial](federated_learning_for_image_classification.ipynb) [anterior](federated_learning_for_text_generation.ipynb) sobre aprendizaje federado.

La media ponderada de referencia del algoritmo [FedAvg](http://proceedings.mlr.press/v54/mcmahan17a/mcmahan17a.pdf) se puede expresar a través de `tff.aggregators.MeanFactory` de la siguiente manera:

```
mean = tff.aggregators.MeanFactory()
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    ...,
    model_aggregator=mean)
```

Las técnicas que sirven para ampliar la media ponderada que se tratan en este tutorial son las siguientes:

- Reducción a cero
- Recorte
- Privacidad diferencial
- Compresión
- Agregación segura

La extensión se realiza mediante composición, donde `MeanFactory` envuelve una fábrica interna a la que delega alguna parte de la agregación, o está envuelta por otra fábrica de agregación. Para obtener más detalles sobre el diseño, consulte el tutorial sobre [implementación de agregadores personalizados](custom_aggregators.ipynb).

Primero, explicaremos cómo habilitar y configurar estas técnicas de forma individual y, luego, mostraremos cómo se pueden combinar entre sí.

## Técnicas

Antes de profundizar en las técnicas individuales, presentamos el algoritmo de coincidencia de cuartiles, que será útil para configurar las técnicas que se describen a continuación.

### Coincidencia de cuartiles

Varias de las técnicas de agregación que se detallan a continuación requieren el uso de un límite de norma que controle algún aspecto de la agregación. Estos límites se pueden proporcionar como una constante, pero generalmente es mejor adaptarlos durante el curso del entrenamiento. Lo que se recomienda es usar el algoritmo de coincidencia de cuartiles de [Andrew et al. (2019)](https://arxiv.org/abs/1905.03871), propuesto inicialmente por su compatibilidad con la privacidad diferencial, pero que es útil en un sentido más amplio. Para estimar el valor en un cuartil específico, se puede usar `tff.aggregators.PrivateQuantileEstimationProcess`. Por ejemplo, para adaptarse a la mediana de una distribución, se puede usar lo que sigue:

In [ ]:
median_estimate = tff.aggregators.PrivateQuantileEstimationProcess.no_noise(
    initial_estimate=1.0, target_quantile=0.5, learning_rate=0.2)

Las diferentes técnicas que utilizan el algoritmo de estimación de cuartiles necesitan diferentes valores de los parámetros del algoritmo, como veremos a continuación. En general, aumentar el parámetro `learning_rate` implica una adaptación más rápida al cuantil correcto, pero con una varianza mayor. El método de clase `no_noise` construye un proceso de coincidencia de cuartiles que no agrega ruido para lograr privacidad diferencial.

### Reducción a cero

La reducción a cero se refiere al método de reemplazo de valores inusualmente grandes por ceros. Aquí, "inusualmente grande" podría significar superior a un umbral predefinido, o grande en comparación con los valores de rondas anteriores del cálculo. La reducción a cero puede aumentar la solidez del sistema ante la corrupción de datos en clientes defectuosos.

Para calcular una media de valores con normas L-infinity mayores que `ZEROING_CONSTANT` reducida a cero, envolvemos `tff.aggregators.MeanFactory` con un agregador `tff.aggregators.zeroing_factory` que ejecuta la reducción a cero:

```
zeroing_mean = tff.aggregators.zeroing_factory(
    zeroing_norm=MY_ZEROING_CONSTANT,
    inner_agg_factory=tff.aggregators.MeanFactory())
```

Aquí, envolvemos una `MeanFactory` con una `zeroing_factory` porque queremos que los efectos (de preagregación) de `zeroing_factory` se apliquen a los valores en los clientes antes de que se pasen a la `MeanFactory` interna para su agregación mediante el promediado.

No obstante, para la mayoría de las aplicaciones, recomendamos la reducción a cero adaptativa con el estimador de cuantiles. Para esto, usamos el algoritmo de coincidencia de cuartiles de la siguiente manera:

In [ ]:
zeroing_norm = tff.aggregators.PrivateQuantileEstimationProcess.no_noise(
    initial_estimate=10.0,
    target_quantile=0.98,
    learning_rate=math.log(10),
    multiplier=2.0,
    increment=1.0)
zeroing_mean = tff.aggregators.zeroing_factory(
    zeroing_norm=zeroing_norm,
    inner_agg_factory=tff.aggregators.MeanFactory())

# Equivalent to:
# zeroing_mean = tff.learning.robust_aggregator(clipping=False)

Los parámetros se eligieron para que el proceso se adapte muy rápidamente (`learning_rate` relativamente grande) a un valor ligeramente superior a los valores más grandes vistos hasta ahora. Para una estimación cuantil `Q`, el umbral que se usa para la reducción a cero será `Q * multiplier + increment`.

### Recorte a la norma de límite L2

Recortar las actualizaciones del cliente (proyectarlas en una bola L2) puede mejorar la solidez ante los valores atípicos. Un `tff.aggregators.clipping_factory` se estructura exactamente como `tff.aggregators.zeroing_factory` que se mencionó anteriormente y puede tomar una constante o un `tff.templates.EstimationProcess` como su argumento `clipping_norm`. La mejor práctica recomendada es utilizar un recorte que se adapte moderadamente rápido a una norma moderadamente alta, de la siguiente manera:

In [ ]:
clipping_norm = tff.aggregators.PrivateQuantileEstimationProcess.no_noise(
    initial_estimate=1.0,
    target_quantile=0.8,
    learning_rate=0.2)
clipping_mean = tff.aggregators.clipping_factory(
    clipping_norm=clipping_norm,
    inner_agg_factory=tff.aggregators.MeanFactory())

# Equivalent to:
# clipping_mean = tff.learning.robust_aggregator(zeroing=False)

Según nuestra experiencia con muchos problemas, el valor preciso de `target_quantile` no parece importar demasiado siempre que las tasas de aprendizaje se ajusten adecuadamente. Sin embargo, configurarlo en un nivel muy bajo podría requerir el aumento de la tasa de aprendizaje del servidor para obtener el mejor rendimiento, en comparación con la falta de recorte, razón por la cual recomendamos 0.8 como valor predeterminado.

### Privacidad diferencial

TFF también admite agregación diferencialmente privada, a través del recorte adaptativo y el ruido gaussiano. Se puede construir un agregador para ejecutar promediados diferenciales privados de la siguiente manera:

In [ ]:
dp_mean = tff.aggregators.DifferentiallyPrivateFactory.gaussian_adaptive(
    noise_multiplier=0.1, clients_per_round=100)

# Equivalent to:
# dp_mean = tff.learning.dp_aggregator(
#   noise_multiplier=0.1, clients_per_round=100, zeroing=False)

Puede encontrar orientación sobre cómo configurar el argumento `noise_multiplier` en el [tutorial de DP en TFF](https://www.tensorflow.org/federated/tutorials/federated_learning_with_differential_privacy).

### Compresión con pérdida

En comparación con la compresión sin pérdidas como gzip, la compresión con pérdidas generalmente da como resultado una relación de compresión mucho mayor y aún se puede combinar con la compresión sin pérdidas posteriormente. Dado que se necesita dedicar menos tiempo a la comunicación entre el cliente y el servidor, las rondas de entrenamiento se completan más rápido. Debido a la naturaleza inherentemente aleatoria de los algoritmos de aprendizaje, hasta cierto umbral, la inexactitud de la compresión con pérdidas no tiene un impacto negativo en el rendimiento general.

La recomendación predeterminada es usar una cuantificación uniforme simple (consulte [Suresh et al.,](http://proceedings.mlr.press/v70/suresh17a/suresh17a.pdf) por ejemplo), parametrizada por dos valores: el `threshold` de compresión del tamaño del tensor y el número de `quantization_bits`. Para cada tensor `t`, si el número de elementos de `t` es menor o igual al `threshold`, no se comprime. Si es mayor, los elementos de `t` se cuantifican mediante redondeo aleatorio a bits `quantizaton_bits`. Es decir, aplicamos la siguiente operación

`t = round((t - min(t)) / (max(t) - min(t)) * (2**quantizaton_bits - 1)),`

lo que da como resultado valores enteros en el rango de `[0, 2**quantizaton_bits-1]`. Los valores cuantificados se empaquetan directamente en un tipo de número entero para su transmisión y luego se aplica la transformación inversa.

Recomendamos establecer `quantizaton_bits` en 8 y `threshold` en 20000:


In [ ]:
compressed_mean = tff.aggregators.MeanFactory(
    tff.aggregators.EncodedSumFactory.quantize_above_threshold(
        quantization_bits=8, threshold=20000))

# Equivalent to:
# compressed_mean = tff.learning.compression_aggregator(zeroing=False, clipping=False)

#### Sugerencias de ajuste

Ambos parámetros, `quantization_bits` y `threshold`, se pueden ajustar, y la cantidad de clientes que participan en cada ronda de entrenamiento también puede afectar la efectividad de la compresión.

**Umbral.** Se elige el valor predeterminado de 20000 porque se ha observado que las variables con una pequeña cantidad de elementos, como los sesgos en los tipos de capas comunes, son mucho más sensibles a la introducción de ruido. Además, en la práctica no se gana mucho al comprimir variables con un número pequeño de elementos, ya que, para empezar, su tamaño sin comprimir es relativamente pequeño.

En algunas aplicaciones, quizá tenga sentido cambiar la elección del umbral. Por ejemplo, los sesgos de la capa de salida de un modelo de clasificación pueden ser más sensibles al ruido. Si está entrenando un modelo de lenguaje con un vocabulario de 20004, tal vez quiera establecer `threshold` en 20004.

**Bits de cuantificación.** El valor predeterminado de 8 para `quantization_bits` debería ser adecuado para la mayoría de los usuarios. Si 8 funciona bien y desea obtener un poco más de rendimiento, puede intentar reducirlo a 7 o 6. Si los recursos permiten hacer una pequeña búsqueda en la cuadrícula, le recomendamos que identifique el valor para el cual el entrenamiento se vuelve inestable o la calidad del modelo final comienza a degradarse y, luego, aumente ese valor en dos. Por ejemplo, si establecer `quantization_bits` en 5 funciona, pero establecerlo en 4 degrada el modelo, recomendamos que el valor predeterminado sea 6 para "ir a lo seguro".

**Clientes por ronda.** Tenga en cuenta que aumentar significativamente el número de clientes por ronda permitiría que un valor más pequeño para `quantization_bits` funcione bien, porque la inexactitud aleatoria introducida por la cuantificación podría compensarse al promediar las actualizaciones de más clientes.

### Agregación segura

Por agregación segura (SecAgg) nos referimos a un protocolo criptográfico en el que las actualizaciones del cliente se cifran de tal manera que el servidor solo puede descifrar su suma. Si el número de clientes que informan es insuficiente, el servidor no aprenderá nada en absoluto; y en ningún caso podrá inspeccionar actualizaciones individuales. Esto se logra con ayuda del operador `tff.federated_secure_sum_bitwidth`.

Las actualizaciones del modelo son valores de coma flotante, pero SecAgg opera con números enteros. Por lo tanto, necesitamos recortar cualquier valor grande hasta cierto límite antes de la discretización a un tipo entero. El límite de recorte puede ser constante o se puede determinar de forma adaptativa (el valor predeterminado recomendado). Luego, los números enteros se suman de forma segura y la suma se vuelve a asignar al dominio de coma flotante.

Para calcular una media con valores ponderados sumados usando SecAgg con `MY_SECAGG_BOUND` como límite de recorte, pase `SecureSumFactory` a `MeanFactory` como:

```
secure_mean = tff.aggregators.MeanFactory(
    tff.aggregators.SecureSumFactory(MY_SECAGG_BOUND))
```

Haga lo mismo mientras determina los límites de forma adaptativa:

In [ ]:
secagg_bound = tff.aggregators.PrivateQuantileEstimationProcess.no_noise(
    initial_estimate=50.0,
    target_quantile=0.95,
    learning_rate=1.0,
    multiplier=2.0)
secure_mean = tff.aggregators.MeanFactory(
    tff.aggregators.SecureSumFactory(secagg_bound))

# Equivalent to:
# secure_mean = tff.learning.secure_aggregator(zeroing=Fasle, clipping=False)

#### Sugerencias de ajuste

Los parámetros adaptativos se eligen de modo que los límites sean estrechos (no perderemos mucha precisión en la discretización), pero el recorte es poco frecuente.

Si se ajustan los parámetros, tenga en cuenta que el protocolo SecAgg suma las actualizaciones ponderadas del modelo, después de ponderar la media. Las ponderaciones suelen ser la cantidad de puntos de datos procesados ​​localmente; por lo tanto, entre diferentes tareas, el límite derecho podría depender de esta cantidad.

No recomendamos que se use el argumento de palabra clave `increment` al crear `secagg_bound` adaptativo, ya que esto podría resultar en una gran pérdida de precisión relativa, en el caso de que la estimación real termine siendo pequeña.

El fragmento de código anterior utilizará SecAgg solo para los valores ponderados. Si también se debe utilizar SecAgg para la suma de ponderaciones, que los límites se establezcan como constantes, ya que, en una configuración de entrenamiento común, la mayor ponderación posible se conocerá de antemano:

```
secure_mean = tff.aggregators.MeanFactory(
    value_sum_factory=tff.aggregators.SecureSumFactory(secagg_bound),
    weight_sum_factory=tff.aggregators.SecureSumFactory(
        upper_bound_threshold=MAX_WEIGHT, lower_bound_threshold=0.0))
```

## Técnicas de composición

Las técnicas individuales para ampliar una media introducida anteriormente se pueden combinar entre sí.

Recomendamos que el orden en el que se aplican estas técnicas en los clientes sea el siguiente:

1. Reducción a cero
2. Recorte
3. Otras técnicas

Los agregadores en el módulo `tff.aggregators` se componen envolviendo "agregadores internos" (cuyos efectos previos a la agregación ocurren al final y los efectos posteriores a la agregación ocurren primero) dentro de "agregadores externos". Por ejemplo, para ejecutar la reducción a cero, el recorte y la compresión (en ese orden), se escribiría de este modo:

```
# Compression is innermost because its pre-aggregation effects are last.
compressed_mean = tff.aggregators.MeanFactory(
    tff.aggregators.EncodedSumFactory.quantize_above_threshold(
        quantization_bits=8, threshold=20000))
# Compressed mean is inner aggregator to clipping...
clipped_compressed_mean = tff.aggregators.clipping_factory(
    clipping_norm=MY_CLIPPING_CONSTANT,
    inner_agg_factory=compressed_mean)
# ...which is inner aggregator to zeroing, since zeroing happens first.
final_aggregator = tff.aggregators.zeroing_factory(
    zeroing_norm=MY_ZEROING_CONSTANT,
    inner_agg_factory=clipped_compressed_mean)
```

Observe que esta estructura coincide con los [agregadores predeterminados](https://github.com/tensorflow/federated/blob/11e4f632b38745c9b38cc39fa1fe67771c206e77/tensorflow_federated/python/learning/model_update_aggregator.py) para los algoritmos de aprendizaje.

También se admiten otras composiciones. Ampliaremos este documento cuando estemos seguros de que podemos proporcionar una configuración predeterminada que funcione en múltiples aplicaciones diferentes. Para implementar nuevas ideas, consulte el tutorial sobre [implementación de agregadores personalizados](custom_aggregators.ipynb).